In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import category_encoders as ce
#from scipy import stats

pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
pd.options.display.float_format = '{:.5f}'.format
%matplotlib inline
#sns.set(style='whitegrid', palette='muted', font_scale=1.5)
import warnings
warnings.filterwarnings('ignore')

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#pd.set_option('display.max_rows', 20)

In [ ]:
!pip install openpyxl

In [ ]:
data = pd.read_excel("/kaggle/input/housing-data/housingData_7R.xlsx")

In [ ]:
#data = df.copy()

In [ ]:
df = data.copy()

In [ ]:
#data = df.copy()

In [ ]:
data.head()

In [ ]:
data.shape

In [ ]:
data.columns

In [ ]:
data = data.drop('Unnamed: 0', axis = 1)

In [ ]:
data.describe(include = "all")

In [ ]:
data['adDescription'].head()

In [ ]:
data['adDealType'].value_counts()

In [ ]:
data['Deal_Type'] = data['adDealType'].copy()

In [ ]:
target_map = {
    10: 'rental',
    20: 'sell'}

data.Deal_Type.replace(target_map, inplace = True)

In [ ]:
data['Deal_Type'].head()

In [ ]:
data.columns

In [ ]:
len(data.columns)

In [ ]:
original_col = data.columns

new_col = ['ID', 'DealType', 'Canton', 'ZipCode',
           'City','PublishedDate', 'AvailableDate', 'PriceText', 
           'Description', 'LangDetected', 'NumRooms', 'Floor', 
           'YearOfConstruction', 'NumApartments', 'Floor1', 
           'NumApartments1', 'LivingSpace', 'BuildingArea', 
           'UsefulArea', 'CoordE', 'CoordN' ,'Deal_Type']

data = data.rename(columns=dict(zip(original_col,new_col)))

data.head()

In [ ]:
def percentage_missing(df):    
    pm = (df.isnull().sum()/len(data))*100
    return pm

In [ ]:
# replace all column names (in place)
#new_cols = [‘column_x’, ‘column_y’, ‘column_z’]
#df.columns = new_cols

## EDA

In [ ]:
data.head()

In [ ]:
data.columns

In [ ]:
sns.set_style('darkgrid')
plt.figure(figsize=(8, 5))
sns.countplot(data['Deal_Type'])
plt.title('Target variable distribution')
plt.show()

In [ ]:
data.shape

In [ ]:
data.isnull().sum().sort_values(ascending = False)

In [ ]:
#Plot missing values in data
ax = data.isna().sum().sort_values().plot(kind = 'barh', figsize = (9, 10))
plt.title('Percentage of Missing Values Per Column Dataset', fontdict={'size':15})
for p in ax.patches:
    percentage ='{:,.0f}%'.format((p.get_width()/data.shape[0])*100)
    width, height =p.get_width(),p.get_height()
    x=p.get_x()+width+0.02
    y=p.get_y()+height/2
    ax.annotate(percentage,(x,y))

In [ ]:
#To find the percentage of missing rows in each column
total = data.isnull().sum().sort_values(ascending=False)
percentage = ((data.isnull().sum()/data.isnull().count()).sort_values(ascending=False)*100)
missing_data = pd.concat([total, percentage], axis=1, keys=['Total', 'Percentage'])
missing_data.head(20)

## Treating Missing Values and Outliers

In [ ]:
data.isnull().sum().sort_values(ascending=False)

In [ ]:
mis_columns = ['NumApartments', 'NumApartments1', 'BuildingArea', 'UsefulArea' ]

In [ ]:
data.corr()

In [ ]:
data['NumApartments'].head()

In [ ]:
data['NumApartments'].value_counts()

In [ ]:
mis_columns

In [ ]:
NumApartments = data['NumApartments'].copy()
BuildingArea = data['BuildingArea'].copy()
UsefulArea = data['UsefulArea'].copy()

In [ ]:
data.columns

In [ ]:
outliers = []
def detect_outliers_zscore(data):
    thres = 3
    mean = np.mean(data)
    std = np.std(data)
    # print(mean, std)
    for i in data:
        z_score = (i-mean)/std
        if (np.abs(z_score) > thres):
            outliers.append(i)
    return outliers# Driver code

#sample_outliers = detect_outliers_zscore(data['adNumApartments.1'])
#print("Outliers from Z-scores method: ", sample_outliers)

In [ ]:
sns.boxplot(data['NumApartments'])
plt.show()

In [ ]:
sns.distplot(data['NumApartments'])
plt.show()

In [ ]:
data = data.drop(columns = mis_columns, axis = 1)

In [ ]:
data.columns

In [ ]:
data.isnull().sum().any()

In [ ]:
data.isnull().sum().sort_values(ascending = False)

In [ ]:
data['City'].value_counts()

In [ ]:
City = data['City'].copy()

In [ ]:
data['City'].describe()

In [ ]:
data['City'].head()

In [ ]:
#Filling Null Values
data['City'].fillna('Zürich', inplace = True) #Fillna
#data['City'].ffill(axis = 0)  #ForwardFill

In [ ]:
data['City'].mode()

In [ ]:
data.isnull().sum().sort_values(ascending = False)

In [ ]:
# ptype_encode = {}
# ptype_encode_values = range(16,0,-1)
# for i,k in zip(type_count.index,ptype_encode_values):
#     ptype_encode[i]=k
# ptype_encode

# data['adCity'] = data['adCity'].map(ptype_encode)

#### **Exploring the City copied from data['City']**

In [ ]:
City.head()

In [ ]:
City = pd.DataFrame(data = City)
City.head()

In [ ]:
City.info()

In [ ]:
encoder=ce.TargetEncoder(cols='City')

In [ ]:
data['City'].head()

In [ ]:
data['City'].head(20)

In [ ]:
data['City'].describe()

In [ ]:
data.describe(include = 'all')

In [ ]:
#sns.scatterplot(x = data['City'], y = data['PriceText'])
#plt.show()

## Exploring the target Column

### PriceText

In [ ]:
PriceText = data['PriceText'].copy()
PriceText = pd.DataFrame(data = PriceText)
PriceText.head()

In [ ]:
data['PriceText'].head()

In [ ]:
data['PriceText'] = data['PriceText'].replace(',','', regex = True)
data['PriceText'].head()

In [ ]:
data['PriceText'] = data['PriceText'].replace('CHF','', regex = True)
data['PriceText'].head()

In [ ]:
data['PriceText'].describe()

In [ ]:
data['PriceText'].isnull().sum()

In [ ]:
data['PriceText'].value_counts()

In [ ]:
data['PriceText'] = data['PriceText'].replace('On request',np.nan)
data['PriceText'].value_counts()

In [ ]:
data['PriceText'].isnull().sum()

In [ ]:
ax = data.isna().sum().sort_values().plot(kind = 'barh', figsize = (9, 10))
plt.title('Percentage of Missing Values Per Column Dataset', fontdict={'size':15})
for p in ax.patches:
    percentage ='{:,.0f}%'.format((p.get_width()/data.shape[0])*100)
    width, height =p.get_width(),p.get_height()
    x=p.get_x()+width+0.02
    y=p.get_y()+height/2
    ax.annotate(percentage,(x,y))

In [ ]:
data['PriceText'] = data['PriceText'].replace('CHF','', regex = True)
data['PriceText'].head()

In [ ]:
data['PriceText'].value_counts()

In [ ]:
data['PriceText'] = data['PriceText'].replace('EUR','', regex = True)
data['PriceText'].head()

In [ ]:
data['PriceText'].dtype

In [ ]:
data['PriceText'].value_counts()

In [ ]:
data['PriceText'] = data['PriceText'].apply(pd.to_numeric)

In [ ]:
data['PriceText'].dtype

In [ ]:
data['PriceText'].describe()

In [ ]:
sns.distplot(data['PriceText'])

In [ ]:
sns.boxplot(data['PriceText'])

In [ ]:
outliers = []
def detect_outliers_zscore(data):
    thres = 3
    mean = np.mean(data)
    std = np.std(data)
    # print(mean, std)
    for i in data:
        z_score = (i-mean)/std
        if (np.abs(z_score) > thres):
            outliers.append(i)
    return outliers# Driver code

In [ ]:
sample_outliers = detect_outliers_zscore(data['PriceText'])
print("Outliers from Z-scores method: ", sample_outliers)

Since, PriceText contains outliers, drop the rows

In [ ]:
data = data[data["PriceText"]<min(sample_outliers)].reset_index(drop = True)

In [ ]:
data['PriceText'].describe()

In [ ]:
sns.distplot(data['PriceText'])

In [ ]:
sample_outliers = detect_outliers_zscore(data['PriceText'])
#print("Outliers from Z-scores method: ", sample_outliers)

In [ ]:
data = data[data["PriceText"]<min(sample_outliers)].reset_index(drop = True)

In [ ]:
data["PriceText"].describe()

In [ ]:
sns.boxplot(data['PriceText'])

In [ ]:
plt.boxplot(data["PriceText"])
plt.title("Detecting outliers using Boxplot")
plt.xlabel('Sample')

In [ ]:
sns.distplot(data['PriceText'])

**Encoding the categorical column [''City''] with the target column**

In [ ]:
City.shape, data['PriceText'].shape, data['City'].shape

In [ ]:
City = data['City'].copy()

In [ ]:
City.shape, data['PriceText'].shape, data['City'].shape

In [ ]:
#Fit and Transform Train Data
City = encoder.fit_transform(City,data['PriceText'])

In [ ]:
City.head()

In [ ]:
City = City.apply(pd.to_numeric)

In [ ]:
sns.distplot(City)
plt.show()

In [ ]:
#City.skew()

In [ ]:
#City.dtype

In [ ]:
plt.boxplot(City)
plt.show()

In [ ]:
data['City'] = City.copy()

In [ ]:
data.isnull().sum()

Dropping the description for now, Will work on it later

In [ ]:
Description = data['Description'].copy()

In [ ]:
data = data.drop('Description', axis = 1)

In [ ]:
data.describe()

In [ ]:
data.info()

In [ ]:
data.columns

In [ ]:
data.shape

In [ ]:
data.columns

### LangDetected

In [ ]:
data['LangDetected'].value_counts()

In [ ]:
obj_col = []

In [ ]:
obj_col.append('LangDetected')

### Canton

In [ ]:
data['Canton'].head()

In [ ]:
data['Canton'].value_counts()

Run it at the later end because it will add to the number of columns and there is still plenty of work to do

In [ ]:
#data = pd.get_dummies(data = data['Canton'], drop_first = True)

In [ ]:
obj_col.append('Canton')

### ZipCode

In [ ]:
data.ZipCode.head()

In [ ]:
data.ZipCode.value_counts()

Is this column really relevant to the model????

In [ ]:
data.columns

## Date_Columns

In [ ]:
date_col = ['PublishedDate', 'AvailableDate']

In [ ]:
data[date_col].head()

In [ ]:
data['AvailableDate'].value_counts()

In [ ]:
data['AvailableDate'].head()

In [ ]:
#sofort = 'right away'
#nach Vereinbarung = by arrangement

In [ ]:
data['PublishedDate'].head()

In [ ]:
data.columns

In [ ]:
data['Description'] = Description.copy()

In [ ]:
data[['PublishedDate', 'AvailableDate', 'DealType', 'PriceText', 'Description']].head()

In [ ]:
data['PublishedDate'].head()

In [ ]:
AvlDate = data['AvailableDate'].copy()
PubDate = data['PublishedDate'].copy()

In [ ]:
#AvlDate = pd.to_datetime(AvlDate)
AvlDate

In [ ]:
AvlDate.value_counts()

In [ ]:
replacement_mapping_dict = {
    "nach Vereinbarung": np.nan,
    "sofort": np.nan,
    "By arrangement": np.nan,
    "Immediately": np.nan,
    "not available": np.nan
}
AvlDate = AvlDate.replace(replacement_mapping_dict)

In [ ]:
AvlDate.value_counts()

In [ ]:
len(data)

In [ ]:
(AvlDate.isnull().sum()/len(data))*100

In [ ]:
#AvlDate.fillna("Others", inplace = True)

In [ ]:
AvlDate.isnull().sum()

In [ ]:
#AvlDate = pd.to_datetime(AvlDate)

In [ ]:
AvlDate.describe()

In [ ]:
percentage_missing(AvlDate)

In [ ]:
AvlDate = pd.DataFrame(AvlDate)

In [ ]:
ax = AvlDate.isna().sum().sort_values().plot(kind = 'barh', figsize = (9, 10))
plt.title('Percentage of Missing Values Per Column Dataset', fontdict={'size':15})
for p in ax.patches:
    percentage ='{:,.0f}%'.format((p.get_width()/data.shape[0])*100)
    width, height =p.get_width(),p.get_height()
    x=p.get_x()+width+0.02
    y=p.get_y()+height/2
    ax.annotate(percentage,(x,y))

Maybe I will need to drop the avilable dates for now because of too much null value

### Pend Pend Pend Pend

### PublishedDate

In [ ]:
data['PublishedDate'].head()

In [ ]:
data['PublishedDate'] = pd.to_datetime(data['PublishedDate'])

In [ ]:
data['PublishedDate'].head()

In [ ]:
#Extracting the year, month & day from the published datetime column

# Year
data['PublishedYear'] = data['PublishedDate'].dt.year

# Month
data['PublishedMonth'] = data['PublishedDate'].dt.month

# Day
data['PublishedDay'] = data['PublishedDate'].dt.day

## We Move to other columns again******

In [ ]:
data.columns

In [ ]:
NumRooms = data['NumRooms'].copy()

In [ ]:
data['NumRooms'].head()

In [ ]:
(data.NumRooms.isnull().sum()/len(data))*100

In [ ]:
data['NumRooms'].value_counts()

In [ ]:
data['NumRooms'] = data['NumRooms'].replace('None', np.nan)
data['NumRooms'].head()

In [ ]:
data['NumRooms'] = data['NumRooms'].apply(pd.to_numeric)

In [ ]:
data['NumRooms'].isnull().sum()

In [ ]:
percentage_missing(data['NumRooms'])

In [ ]:
data['NumRooms'].value_counts()

In [ ]:
data['NumRooms'].dtype

In [ ]:
sns.boxplot(data['NumRooms'])

In [ ]:
data['NumRooms'].describe()

In [ ]:
def detect_outliers_zscore(data):
    thres = 3
    mean = np.mean(data)
    std = np.std(data)
    # print(mean, std)
    for i in data:
        z_score = (i-mean)/std
        if (np.abs(z_score) > thres):
            outliers.append(i)
    return outliers# Driver code

In [ ]:
sample_outliers = detect_outliers_zscore(data['NumRooms'])
#print("Outliers from Z-scores method: ", sample_outliers)

In [ ]:
data = data[data["NumRooms"]<min(sample_outliers)].reset_index(drop = True)

In [ ]:
sns.boxplot(data['NumRooms'])
plt.title("Detecting outliers using Boxplot")
plt.xlabel('Sample')

In [ ]:
data['NumRooms'].describe()

In [ ]:
sns.distplot(data['NumRooms'])

In [ ]:
data['NumRooms'].isnull().sum()

In [ ]:
#data['NumRooms'].groupby()

In [ ]:
percentage_missing(data['NumRooms'])

In [ ]:
data.columns

### Floor Column

In [ ]:
data[['Floor', 'Floor1']].head()

In [ ]:
data = data.drop('Floor1', axis = 1)

In [ ]:
Floor = data['Floor'].copy()

In [ ]:
data['Floor'].value_counts()

In [ ]:
#To remove the digits from the floor column
data['Floor'] = data['Floor'].replace('\d+', '', regex = True)

In [ ]:
data['Floor'].value_counts()

* untergeschoss = basement 
* Stock oder höher = floor or higher

* This is the equivalent of this words in English

In [ ]:
# The replace fucntion to replace the words in german in the english format and correct the format of the incorrect value
replacement_mapping_dict = {
    ". Stock": 'Stock',
    ". floor ": 'Floor',
    "Untergeschoss": 'basement',
    ". Stock oder höher": 'floor or higher',
    "EG": 'ground floor'
}
data['Floor'] = data['Floor'].replace(replacement_mapping_dict)

In [ ]:
data['Floor'].value_counts()

In [ ]:
obj_col.append('Floor')

In [ ]:
data[['Floor', 'NumRooms']].head()

In [ ]:
data.columns

## Things to try out

* **data = data.sort_values(by=['adPublishedDate']).reset_index(drop=True)**
* Trying to sort the data by dates... 'adPublishedDate', 'adAvailableDate'
* How many days is between the published date and available date
* Extract days and month from the date columns
* Get the number of years between the year of construction and the Available Year of the house & Year of Publishing the year for an ad

### Year of Construction

In [ ]:
Year = data['YearOfConstruction'].copy()

In [ ]:
data['YearOfConstruction'].head()

In [ ]:
data['YearOfConstruction'].value_counts()

In [ ]:
data['YearOfConstruction'] = data['YearOfConstruction'].replace('not available', 0)

In [ ]:
data['YearOfConstruction'] = data['YearOfConstruction'].apply(pd.to_numeric)

In [ ]:
data['YearOfConstruction'].value_counts()

In [ ]:
sns.distplot(data['YearOfConstruction'])

In [ ]:
sns.boxplot(data['YearOfConstruction'])
plt.show()

In [ ]:
data['YearOfConstruction'].describe()

In [ ]:
data.columns

In [ ]:
data['YearOfConstruction'].describe()

### Living Space

In [ ]:
LivingSpace = data['LivingSpace'].copy()

In [ ]:
data['LivingSpace'].head()

In [ ]:
data['LivingSpace'].dtype

In [ ]:
data['LivingSpace'].describe()

In [ ]:
data['LivingSpace'].value_counts()

In [ ]:
data['LivingSpace'] = data['LivingSpace'].replace('not available', 0)
data['LivingSpace'].value_counts()

In [ ]:
data['LivingSpace'] = data['LivingSpace'].replace(',', '', regex=True)
data['LivingSpace'].value_counts()

In [ ]:
sns.distplot(data['LivingSpace'])

In [ ]:
data['LivingSpace'] = data['LivingSpace'].apply(pd.to_numeric)

In [ ]:
sns.boxplot(data['LivingSpace'])
plt.show()

In [ ]:
data['LivingSpace'].describe()

In [ ]:
sample_outliers = detect_outliers_zscore(data['LivingSpace'])

In [ ]:
#print(sample_outliers)

In [ ]:
#data = data[data["LivingSpace"]<min(sample_outliers)].reset_index(drop = True)

In [ ]:
data['LivingSpace'].describe()

In [ ]:
#sample_outliers = detect_outliers_zscore(data['LivingSpace'])
#print(sample_outliers)

In [ ]:
sns.boxplot(data['LivingSpace'])

In [ ]:
sns.distplot(data['LivingSpace'])

In [ ]:
data['LivingSpace'].describe()

In [ ]:
Livingspace = data['LivingSpace'].copy()

In [ ]:
Livingspace = Livingspace.apply(pd.to_numeric)
Livingspace = pd.DataFrame(Livingspace)
Livingspace.head()

In [ ]:
Livingspace['LivingSpace'].head()

In [ ]:
Livingspace.shape

In [ ]:
sample_outliers = detect_outliers_zscore(Livingspace['LivingSpace'])
#print(sample_outliers)

In [ ]:
#data = data[data["LivingSpace"]<min(sample_outliers)].reset_index(drop = True)
Livingspace = Livingspace[Livingspace["LivingSpace"]<min(sample_outliers)].reset_index(drop = True)

In [ ]:
Livingspace.shape

In [ ]:
LivingSpace.shape

Should you fix the outliers or leave it in this scenerio???

In [ ]:
data.columns

#### Some Columns with outliers that you can work on maybe it would improve the model
* YearOfObservation
* LivingSpace

### Coordinates

In [ ]:
data[['CoordN', 'CoordE']].head()

In [ ]:
fig, ax = plt.subplots(figsize=(16,8))
ax.scatter(data['CoordE'], data['CoordN'])
ax.set_xlabel('Eastern Coordinates')
ax.set_ylabel('Northern Coordinates')
plt.show()

In [ ]:
data[['CoordN', 'CoordE']].value_counts()

In [ ]:
data['CoordN'].dtype

## Final Preparation for Modelling

In [ ]:
data.columns

In [ ]:
data[['DealType', 'Deal_Type']].head()

In [ ]:
target_map = {
    10: 0,
    20: 1}

In [ ]:
data['DealType'].replace(target_map, inplace = True)
data['DealType'].head()

In [ ]:
data = data.drop('Deal_Type', axis = 1)

In [ ]:
data.columns

In [ ]:
data.shape

In [ ]:
obj_col

In [ ]:
data.columns

In [ ]:
drop_cols = ['ID','AvailableDate', 'Description', 'PublishedDate']

In [ ]:
data.head()

In [ ]:
d = data.copy()

In [ ]:
data = pd.get_dummies(data = data,columns = obj_col, drop_first = True)

In [ ]:
data.head()

In [ ]:
#Remember this for some visualization
#count_plot = sns.catplot(x="adNumApartments.1", kind="count" ,data=data, hue="DealType")
#plt.hist(data['adNumApartments.1'])
#plt.show()

In [ ]:
data.isnull().sum()

In [ ]:
data = data.drop(columns = drop_cols, axis = 1)

In [ ]:
data.shape

In [ ]:
data.info()

In [ ]:
data.describe()

# Modelling

In [ ]:
y = data['PriceText']
X = data.drop(['PriceText'], axis = 1)

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)
x_train.shape, y_train.shape, x_test.shape, y_test.shape

In [ ]:
from sklearn.model_selection import KFold, cross_val_score, StratifiedKFold, train_test_split
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

In [ ]:
#Initialize the model
lr = LinearRegression()
gbr = GradientBoostingRegressor()
rfr = RandomForestRegressor()
xgb = XGBRegressor()
ctb = CatBoostRegressor(silent=True)
lgb = LGBMRegressor(n_estimators=4000, learning_rate=0.1, subsample=0.3, max_depth=-1, min_child_samples=30, colsample_bytree=1.0)

In [ ]:
#train the model
lr.fit(x_train, y_train)
gbr.fit(x_train, y_train)
rfr.fit(x_train, y_train)

xgb.fit(x_train, y_train)
ctb.fit(x_train, y_train)
lgb.fit(x_train, y_train)

In [ ]:
lr_pred = lr.predict(x_test)
gbr_pred = gbr.predict(x_test)
rfr_pred = rfr.predict(x_test)
xgb_pred = xgb.predict(x_test)
ctb_pred = ctb.predict(x_test)
lgb_pred = lgb.predict(x_test)

In [ ]:
lr_rms = np.sqrt(mean_squared_error(y_test, lr_pred))
gbr_rms = np.sqrt(mean_squared_error(y_test, gbr_pred))
rfr_rms = np.sqrt(mean_squared_error(y_test, rfr_pred))
xgb_rms = np.sqrt(mean_squared_error(y_test, xgb_pred))
ctb_rms = np.sqrt(mean_squared_error(y_test, ctb_pred))
lgb_rms = np.sqrt(mean_squared_error(y_test, lgb_pred))

In [ ]:
lr_tr_score = lr.score(x_train, y_train)
gbr_tr_score = gbr.score(x_train, y_train)
rfr_tr_score = rfr.score(x_train, y_train)
xgb_tr_score = xgb.score(x_train, y_train)
ctb_tr_score = ctb.score(x_train, y_train)
lgb_tr_score = lgb.score(x_train, y_train)

In [ ]:
lr_te_score = lr.score(x_test, y_test)
gbr_te_score = gbr.score(x_test, y_test)
rfr_te_score = rfr.score(x_test, y_test)
xgb_te_score = xgb.score(x_test, y_test)
ctb_te_score = ctb.score(x_test, y_test)
lgb_te_score = lgb.score(x_test, y_test)

In [ ]:
# 'Train Score': [lr_tr_score, gbr_tr_score, rfr_tr_score, xgb_tr_score, ctb_tr_score, lgb_tr_score],
# 'Test Score': [lr_te_score, gbr_te_score, rfr_te_score, xgb_te_score, ctb_te_score, lgb_te_score]

In [ ]:
dict_ = {'Algorithm':['LinearRegression', 'GradientBoostingRegressor', 'RandomForestRegressor', 'XGBoost', 'CatBoost', 'LightGBM'], 
         'RMSE_Score':[lr_rms, gbr_rms, rfr_rms, xgb_rms, ctb_rms, lgb_rms]}

df = pd.DataFrame(dict_, index=[0,1,2,3,4,5])
df

In [ ]:
dict1 = {'Algorithm':['LinearRegression', 'GradientBoostingRegressor', 'RandomForestRegressor', 'XGBoost', 'CatBoost', 'LightGBM'], 
         'RMSE_Score':[lr_rms, gbr_rms, rfr_rms, xgb_rms, ctb_rms, lgb_rms],
        'Train Score': [lr_tr_score, gbr_tr_score, rfr_tr_score, xgb_tr_score, ctb_tr_score, lgb_tr_score],
        'Test Score': [lr_te_score, gbr_te_score, rfr_te_score, xgb_te_score, ctb_te_score, lgb_te_score]}

df = pd.DataFrame(dict1, index=[0,1,2,3,4,5])
df

In [ ]:
lgb.fit(x_train, y_train,eval_set=[(x_test,y_test),(x_train,y_train)], verbose=1, early_stopping_rounds=100)

In [ ]:
errcb2=[]
y_pred_totcb2=[]
fold=StratifiedKFold(n_splits = 10)
i=1
for train_index, test_index in fold.split(X,y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    m2  = CatBoostRegressor(n_estimators = 1000, eval_metric = 'RMSE', od_wait = 200, learning_rate = 0.2,
                              bootstrap_type = 'Bernoulli' ,metric_period = 100, max_depth = 8,
                             use_best_model = True)
    m2.fit(X_train,y_train,eval_set=[(X_train,y_train),(X_test, y_test)], early_stopping_rounds = 200, verbose = 100,)
    preds=m2.predict(X_test)
    print("RMSE: ", np.sqrt(mean_squared_error(y_test,preds)))
    errcb2.append((np.sqrt(mean_squared_error(y_test,preds))))
    #np.sqrt(mean_squared_error(y_test, lr_pred)
    #p2 = m2.predict(df_test)
    #y_pred_totcb2.append(p2)
np.mean(errcb2)

In [ ]:
errcb2=[]
y_pred_totcb2=[]
fold=StratifiedKFold(n_splits = 10)
i=1
for train_index, test_index in fold.split(X,y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    m2  = CatBoostRegressor(n_estimators = 4000, eval_metric = 'RMSE', od_wait = 200, learning_rate = 0.1,
                              bootstrap_type = 'Bernoulli' ,metric_period = 100, max_depth = 8,
                             use_best_model = True)
    m2.fit(X_train,y_train,eval_set=[(X_train,y_train),(X_test, y_test)], early_stopping_rounds = 200, verbose = 100,)
    preds=m2.predict(X_test)
    print("RMSE: ", np.sqrt(mean_squared_error(y_test,preds)))
    errcb2.append((np.sqrt(mean_squared_error(y_test,preds))))
    #np.sqrt(mean_squared_error(y_test, lr_pred)
    #p2 = m2.predict(df_test)
    #y_pred_totcb2.append(p2)
np.mean(errcb2)

In [ ]:
metric = make_scorer(mean_squared_error, greater_is_better = True)

In [ ]:
gbr_cv = cross_val_score(gbr, X, y, scoring = metric, cv = 5)
gbr_cv = [np.around(i, decimals=3) for i in gbr_cv]

rfr_cv = cross_val_score(rfr, X, y, scoring = metric, cv = 5)
rfr_cv = [np.around(i, decimals=3) for i in rfr_cv]

lr_cv = cross_val_score(lr, X, y, scoring = metric, cv = 5)
lr_cv = [np.around(i, decimals=3) for i in lr_cv]

ctb_cv = cross_val_score(ctb, X, y, scoring = metric, cv = 5)
ctb_cv = [np.around(i, decimals=3) for i in ctb_cv]


xgb_cv = cross_val_score(xgb, X, y, scoring = metric, cv = 5)
xgb_cv = [np.around(i, decimals=3) for i in xgb_cv]

lgb_cv = cross_val_score(lgb, X, y, scoring = metric, cv = 5)
lgb_cv = [np.around(i, decimals=3) for i in lgb_cv]

In [ ]:
gbr_cv1 = np.sqrt(gbr_cv)
rfr_cv1 = np.sqrt(rfr_cv)
lr_cv1 = np.sqrt(lr_cv)
xgb_cv1 = np.sqrt(xgb_cv)
ctb_cv1 = np.sqrt(ctb_cv)
lgb_cv1 = np.sqrt(lgb_cv)

In [ ]:
gbr_cv1 = [np.around(i, decimals=3) for i in gbr_cv1]
rfr_cv1 = [np.around(i, decimals=3) for i in rfr_cv1]
lr_cv1 = [np.around(i, decimals=3) for i in lr_cv1]
xgb_cv1 = [np.around(i, decimals=3) for i in xgb_cv1]
ctb_cv1 = [np.around(i, decimals=3) for i in ctb_cv1]
lgb_cv1 = [np.around(i, decimals=3) for i in lgb_cv1]

In [ ]:
dict_ = {'Algorithm': ['GradientBoostingRegressor', 'RandomForestRegressor', 'LogisticRegression', 'XGBoost', 'CatBoost', 'LightGBM'], 
         'cv_score':[gbr_cv1, rfr_cv1, lr_cv1, xgb_cv1, ctb_cv1, lgb_cv1], 
         'cv_mean':[np.mean(gbr_cv1), np.mean(rfr_cv1), np.mean(lr_cv1), np.mean(xgb_cv1), np.mean(ctb_cv1), np.mean(lgb_cv1)], 
         'cv_std':[np.std(gbr_cv1), np.std(rfr_cv1), np.std(lr_cv1), np.std(xgb_cv1), np.std(ctb_cv1), np.std(lgb_cv1)]
        }
df = pd.DataFrame(dict_, index = [0,1,2,3,4,5])
df